# Fuzzy C-Means Clustering

In [4]:
# Library imports
import numpy as np
import pandas as pd
import math

In [26]:
# Data imports
data1 = pd.read_csv("data/data1.csv")
data2 = pd.read_csv("data/data2.csv")
data3 = pd.read_csv("data/data3.csv")
data4 = pd.read_csv("data/data4.csv")

In [29]:
## Definition of constants
CONST_M = 1.4


In [22]:
## Definition of general fuctions

def distance(point1, point2, to_power2=False):
    assert len(point1) == len(point2), "Points dimentions are different."
    
    dist = 0;
    for i in range(len(point1)):
        dist = dist + (point1[i] - point2[i])**2
        
    return dist if to_power2 else math.sqrt(dist)



# First Step

We